<a href="https://colab.research.google.com/github/ErisonBarros/Analise-Exploratria-Setores-Censitarios/blob/master/C%C3%B3pia_de_intro_ml_r_hiererquico_worcap2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<img src="https://suspicious-wescoff-e06084.netlify.app/banner_notebook.png">
    
</center>

<br>

<h1> Exemplo de aplicação do Método Hierarquico</h1>

- **Instrutores**: Adriano, Felipe Carvalho e Felipe Menino

- **Realização**: Dia 15/09

- **Descrição**: Objetiva-se apresentar aos alunos exemplos de aplicação de algoritmos de agrupamento 


* **Sumário**:
    * [Descrição do conjunto de dados](#description-dataset)
    * [Leitura dos dados](#import)
    * [Aplicação do **Método Aglomerativo**](#alg)
    * [Visualização dos Dendrograma](#dendro)
    * [Referências](#ref)



* **Links Úteis**:
    * [Livro **Introdução ao Machine Learning**](https://dataat.github.io/introducao-ao-machine-learning/)
    * [Exemplo de **Classificação** em Python](https://www.kaggle.com/phelpsmemo/intro-ml-python-knn-worcap2020)
    * [Exemplo de **Regressão** com **Máquina de Vetores de Suporte** em Python](https://www.kaggle.com/lordadriano/intro-ml-python-svr-worcap2020)
    * [Exemplo de **Agrupamento Kmeans** em R](https://www.kaggle.com/oldlipe/intro-ml-r-kmeans-worcap2020)



## Agrupando dados de países <a id="description-dataset"></a>

<img src="https://images.unsplash.com/photo-1526778548025-fa2f459cd5c1?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1333&q=80" align="right" width="300"/>

Neste *notebook* vamos apresentar um exemplo de aplicação do **Kmeans** usando a linguagem `R`. Para isso, vamos usar os dados sobre os [**Países do mundo**](https://www.kaggle.com/fernandol/countries-of-the-world). 

O conjunto de dados de [**Países do mundo**](https://www.kaggle.com/fernandol/countries-of-the-world) contém informações sobre o número de habitantes, a área do país, renda per capita, entre outros. No entanto, para fins didáticos, estamos usando o dado tratado e limpo, deste repositório: [Machine Learning aplicado a dados espaciais](https://raw.githubusercontent.com/dataAt/ml-aplicado-dados-espacial/master/src/metodos-nao-supervisionados/dados/world_countries). A pergunta que queremos responder neste agrupamento é: `Será que conseguimos obter grupos de países que possuem características semelhantes?` Tudo indica que sim, né? Mas, vamos ver que essa tarefa não é nada trivial!!

# Mão na massa

<br>

> Chega de falar e vamos para o código!!!

### 1. Importação dos pacotes <a id="import"></a>


In [ ]:
library(dplyr)      # Conjunto de funções para manipulação de dados
library(readr)      # Métodos para a leitura dos dados
library(magrittr)   # Operador pipe
library(cluster)    # Algoritmos de agrupamento
library(FactoMineR) # Pacote para análises multivariadas
library(factoextra) # Métodos de visualização para os algoritmos de agrupamentos
library(gridExtra)  # Funções de visualização
library(psych)      # Funções de visualização

### 2. Leitura do conjunto de dados

In [ ]:
# Leitura do conjunto de dados
world_countries <- 
readr::read_csv("https://raw.githubusercontent.com/dataAt/ml-aplicado-dados-espacial/master/src/metodos-nao-supervisionados/dados/world_countries/world_country_clean.csv")

#### 2.1 Seleção dos atributos <a id="description"></a>


In [ ]:
# Seleção dos atributos
world_countries_data <- world_countries %>% 
    dplyr::select(phones_per_1000, infant_mortality_per_1000_births, gdp_per_capita, birthrate, deathrate, climate, literacy_percent) %>%
    scale()

# Alteração no nome das linhas
row.names(world_countries_data) <- world_countries$country 

# Visualização das cinco primeiras linhas
head(world_countries_data, 5)

#### Descrição dos atributos

O conjunto de dados World Countries utilizando nesta seção, possui os seguintes atributos:

- **phones_per_1000:** Telefones a 1000 pessoas
- **infant_mortality_per_1000_births:** Taxa de mortalidade infantil a cada 1000 nascidos
- **gdp_per_capita:** GDP per capita
- **birthrate:** Taxa de natalidade
- **deathrate:** Taxa de mortalidae
- **climate:** Clima
- **literacy_percent:** Porcentagem de alfabetização

#### 2.2 Visualização dos dados

In [ ]:
options(repr.plot.width=25, repr.plot.height=15)

# Criação de um gráfico de correlações
pairs.panels(world_countries_data, 
             method = "pearson", # Método de correlação
             hist.col = "#00AFBB",
             density = TRUE,     # Exibe a densidade
             ellipses = TRUE     # Mostra as elipses de correlações
             )

> É possível observar que muitas das correlações são negativas, a melhor correlação é dada pela taxa de telefone a cada 1000 habitantes pelo GDP per capita. Vamos ver que tipos de relaçãoes podemos encontrar aplicando o método aglomerativo.

### 3. Aplicação do Método Aglomerativo <a id="alg"></a>


#### 3.1 Criação da Matriz de distância

In [ ]:
# Matriz de distância com base na distância euclidiana
world_countries_dist <- dist(world_countries_data, method = "euclidian")

#### 3.2 Criação do agrupamento

In [ ]:
# Criação do agrupamento aglomerativo pelo método de ligação completa com base na distância eucliana
hcut_complete <- hcut(world_countries_data, k = 1, stand = TRUE, hc_method = "complete", hc_func = "hclust", hc_metric = "euclidean")

In [ ]:
# Criação do agrupamento aglomerativo pelo método de ligação simples com base na distância eucliana
hcut_single <- hcut(world_countries_data, k = 1, stand = TRUE, hc_method = "single", hc_func = "hclust", hc_metric = "euclidean")

In [ ]:
# Criação do agrupamento aglomerativo pelo método de ligação pela média com base na distância eucliana
hcut_avg <- hcut(world_countries_data, k = 1, stand = TRUE, hc_method = "average", hc_func = "hclust", hc_metric = "euclidean")

#### 3.3 Visualização dos dendrogramas

In [ ]:
# Visuaização do agrupamento por ligação completa
viz_completa <- factoextra::fviz_dend(hcut_complete, color_labels_by_k = FALSE, rect = FALSE, cex = 0.8, horiz = FALSE, rect_fill = FALSE, palette = "npg") + ggtitle("Agrupamento por ligação completa")

In [ ]:
# Visuaização do agrupamento por ligação simples
viz_single <- factoextra::fviz_dend(hcut_single, color_labels_by_k = FALSE, rect = FALSE, cex = 0.8, horiz = FALSE, palette = "npg") + ggtitle("Agrupamento por ligação simples")

In [ ]:
# Visuaização do agrupamento por ligação média
viz_media <- factoextra::fviz_dend(hcut_avg, color_labels_by_k = FALSE, rect = FALSE, cex = 0.8, horiz = FALSE, palette = "npg", ) + ggtitle("Agrupamento por ligação pela média") 

In [ ]:
options(repr.plot.width=28, repr.plot.height=22)

# Visualização dos agrupamentos
grid.arrange(viz_completa, viz_single, viz_media, nrow = 3, ncol = 1)

> Diversas informações podem ser extraídos com base no dendrograma. Por exemplo, o país Suazilândia aparece somente em um grupo na ligação simples, ou em níveis bem próximos da raiz.

### 4. Cortes nos dendrogramas <a id="dendro"></a>


#### 4.1 Criação dos agrupamentos

> Desta vez vamos usar método de ligação por média

In [ ]:
# Criação do agrupamento aglomerativo pelo método de ligação pela média com base na distância eucliana
hcut_avg_3 <- hcut(world_countries_data, k = 3, stand = TRUE, hc_method = "average", hc_func = "hclust", hc_metric = "euclidean")

In [ ]:
# Criação do agrupamento aglomerativo pelo método de ligação pela média com base na distância eucliana
hcut_avg_5 <- hcut(world_countries_data, k = 5, stand = TRUE, hc_method = "average", hc_func = "hclust", hc_metric = "euclidean")

In [ ]:
# Criação do agrupamento aglomerativo pelo método de ligação pela média com base na distância eucliana
hcut_avg_7 <- hcut(world_countries_data, k = 7, stand = TRUE, hc_method = "average", hc_func = "hclust", hc_metric = "euclidean")

#### 4.2 Visualização dos dendrogramas

In [ ]:
# Visualização do dendrograma com 3 cortes
viz_media_3 <- factoextra::fviz_dend(hcut_avg_3, rect = TRUE,  k_colors = c("#00AFBB","#2E9FDF", "#E7B800", "#FC4E07", "#000000"), 
                                     cex = 0.8, horiz = FALSE, palette = "npg") + ggtitle("Agrupamento por ligação pela média - (K = 3)")

In [ ]:
# Visualização do dendrograma com 5 cortes
viz_media_5 <- factoextra::fviz_dend(hcut_avg_5, rect = TRUE, k_colors = c("#00AFBB","#2E9FDF", "#E7B800", "#FC4E07", "#000000"), 
                                     cex = 0.8, horiz = FALSE, palette = "npg") + ggtitle("Agrupamento por ligação pela média - (K = 5)") 


In [ ]:
# Visualização do dendrograma com 7 cortes
viz_media_7 <- factoextra::fviz_dend(hcut_avg_7, rect = TRUE, k_colors = c("#00AFBB","#2E9FDF", "#E7B800", "#FC4E07", "#000000", "#487eb0", "#353b48"), 
                                     cex = 0.8, horiz = FALSE, palette = "npg", ) + ggtitle("Agrupamento por ligação pela média - (K = 7)") 

In [ ]:
options(repr.plot.width=28, repr.plot.height=22)

# Visualização dos agrupamentos
grid.arrange(viz_media_3, viz_media_5, viz_media_7, nrow = 3, ncol = 1)

> É interessante observar que em grupos existem diversos países do mesmo continente. Por exemplo, o agrupamento com **K = 7**, em que diversos países do continente Africano pertencem ao mesmo grupo.

### Exercício

Compare os agrupamentos Kmeans e hierárquico e aponte as prinpais diferenças.

### Referências <a id="ref"></a>

- https://www.machinegurning.com/rstats/keras/
- http://www.sthda.com/english/wiki/factoextra-r-package-easy-multivariate-data-analyses-and-elegant-visualization
- http://www.milbo.org/rpart-plot/prp.pdf
- https://uc-r.github.io/kmeans_clustering
- https://rpkgs.datanovia.com/factoextra/reference/hkmeans.html